# Data Wrangling for datasets | ADSE 2025

### Correlate Anemia Prevalence, Poverty level, and Malnutrition prevalence

#### Anemia Prevalence, 5 and under

In [97]:
Anemia_U5 = pd.read_csv(r'C:\Users\Leon Pascual\Documents\GitHub\CPE311_Computational-Thinking-with-Python\Final\ADSE\datasets\API_SH.ANM.CHLD.ZS_DS2_en_csv_v2_104143\PVA_U5.csv', skiprows=4)
Anemia_U5.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,54.172067,54.293680,54.361880,54.445964,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,43.900000,44.200000,44.500000,44.900000,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,71.500043,70.826525,69.992481,69.070796,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,61.900000,62.100000,62.300000,62.400000,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
# drop empty columns
Anemia_U5 = Anemia_U5.drop(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999'], axis=1)

In [99]:
southeast_asia = [
    "Brunei Darussalam",
    "Cambodia",
    "Indonesia",
    "Lao PDR",
    "Malaysia",
    "Myanmar",
    "Philippines",
    "Singapore",
    "Thailand",
    "Timor-Leste",
    "Viet Nam"
]
# filter SEA countries
SEA_Anemia_U5 = Anemia_U5.query("`Country Name` in @southeast_asia")
SEA_Anemia_U5.head(15)

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
31,Brunei Darussalam,BRN,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,17.0,16.7,16.6,16.7,16.6,16.7,...,18.2,18.8,19.3,20.0,NaN,NaN,NaN,NaN,NaN,NaN
106,Indonesia,IDN,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,43.9,43.0,42.1,41.2,40.5,39.8,...,37.8,38.0,38.2,38.4,NaN,NaN,NaN,NaN,NaN,NaN
123,Cambodia,KHM,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,65.9,64.8,63.6,62.4,61.3,60.3,...,50.9,50.3,49.6,49.0,NaN,NaN,NaN,NaN,NaN,NaN
129,Lao PDR,LAO,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,50.2,49.3,48.4,47.7,46.9,46.2,...,41.4,41.3,41.3,41.4,NaN,NaN,NaN,NaN,NaN,NaN
160,Myanmar,MMR,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,57.5,56.6,55.8,55.0,54.2,53.5,...,49.6,49.6,49.6,49.6,NaN,NaN,NaN,NaN,NaN,NaN
169,Malaysia,MYS,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,20.2,19.8,19.6,19.4,19.4,19.5,...,22.5,23.1,23.8,24.6,NaN,NaN,NaN,NaN,NaN,NaN
187,Philippines,PHL,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,35.0,34.0,32.9,31.8,30.6,29.2,...,14.7,14.2,13.8,13.5,NaN,NaN,NaN,NaN,NaN,NaN
208,Singapore,SGP,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,11.4,11.2,11.1,11.1,11.1,11.1,...,12.6,12.9,13.2,13.5,NaN,NaN,NaN,NaN,NaN,NaN
233,Thailand,THA,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,26.5,26.2,26.0,25.7,25.4,25.2,...,24.5,24.6,24.7,24.9,NaN,NaN,NaN,NaN,NaN,NaN
237,Timor-Leste,TLS,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,51.3,49.9,48.7,47.7,46.8,46.3,...,46.0,46.1,46.1,46.3,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
# melt dataframe to merge column values together
SEA_Anemia_U5_melted = SEA_Anemia_U5.melt(id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"], 
                                   var_name="Year", 
                                   value_name="Value"
                                   )

SEA_Anemia_U5_melted = SEA_Anemia_U5_melted[SEA_Anemia_U5_melted["Year"].str.isdigit()]
SEA_Anemia_U5_melted["Year"] = SEA_Anemia_U5_melted["Year"].astype(int)

SEA_Anemia_U5_melted.sort_values(by=["Year","Country Name"])
SEA_Anemia_U5_melted.head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Brunei Darussalam,BRN,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,2000,17.0
1,Indonesia,IDN,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,2000,43.9
2,Cambodia,KHM,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,2000,65.9
3,Lao PDR,LAO,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,2000,50.2
4,Myanmar,MMR,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,2000,57.5


In [102]:
SEA_Anemia_U5_melted['Year_Date'] = pd.to_datetime(SEA_Anemia_U5_melted['Year'].astype(str) + '-01-01')


In [103]:
SEA_Anemia_U5_melted.to_csv(r'C:\Users\Leon Pascual\Documents\GitHub\CPE311_Computational-Thinking-with-Python\Final\ADSE\datasets\exported\PVA_U5_melted.csv', index=False)

#### Anemia Prevalence, women of reproductive age

In [63]:
Anemia_Women = pd.read_csv(r'C:\Users\Leon Pascual\Documents\GitHub\CPE311_Computational-Thinking-with-Python\Final\ADSE\datasets\PVA_WRA_PREG.csv')
Anemia_Women.head()

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,AMR,Americas,Country,GTM,Guatemala,Year,2023,...,NaN,NaN,5.2,NaN,17.9,10.0 [5.2-17.9],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
1,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,AMR,Americas,Country,USA,United States of America,Year,2023,...,NaN,NaN,4.7,NaN,18.5,10.2 [4.7-18.5],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
2,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,AMR,Americas,Country,GTM,Guatemala,Year,2023,...,NaN,NaN,5.6,NaN,18.4,10.5 [5.6-18.4],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
3,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,WPR,Western Pacific,Country,AUS,Australia,Year,2023,...,NaN,NaN,4.0,NaN,22.6,11.0 [4.0-22.6],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
4,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,WPR,Western Pacific,Country,AUS,Australia,Year,2023,...,NaN,NaN,4.2,NaN,22.6,11.1 [4.2-22.6],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z


In [ ]:
southeast_asia = [
    "Brunei Darussalam",
    "Cambodia",
    "Indonesia",
    "Lao PDR",
    "Malaysia",
    "Myanmar",
    "Philippines",
    "Singapore",
    "Thailand",
    "Timor-Leste",
    "Viet Nam"
]
# filter SEA countries
SEA_Anemia_Women = Anemia_Women.query("`Location` in @southeast_asia")

In [ ]:
# rename columns
SEA_Anemia_Women = SEA_Anemia_Women.rename(columns={
                                                    'Dim1': 'Sex',
                                                    'Dim2': 'Pregnancy Status',
                                                })
# sort values
SEA_Anemia_Women.sort_values(by=['Period','Location','Pregnancy Status'], ascending=True)


,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
13441,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,WPR,Western Pacific,Country,BRN,Brunei Darussalam,Year,2000,...,NaN,NaN,7.5,NaN,33.3,17.6 [7.5-33.3],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
13674,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,WPR,Western Pacific,Country,BRN,Brunei Darussalam,Year,2000,...,NaN,NaN,21.6,NaN,38.8,30.5 [21.6-38.8],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
13448,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,WPR,Western Pacific,Country,BRN,Brunei Darussalam,Year,2000,...,NaN,NaN,8.3,NaN,33.4,18.2 [8.3-33.4],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
13853,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,WPR,Western Pacific,Country,KHM,Cambodia,Year,2000,...,NaN,NaN,33.9,NaN,57.8,45.6 [33.9-57.8],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
13922,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,WPR,Western Pacific,Country,KHM,Cambodia,Year,2000,...,NaN,NaN,47.3,NaN,64.8,56.6 [47.3-64.8],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
13859,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,WPR,Western Pacific,Country,KHM,Cambodia,Year,2000,...,NaN,NaN,35.0,NaN,58.1,46.4 [35.0-58.1],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
13698,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,SEAR,South-East Asia,Country,IDN,Indonesia,Year,2000,...,NaN,NaN,23.4,NaN,41.4,31.3 [23.4-41.4],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
13813,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,SEAR,South-East Asia,Country,IDN,Indonesia,Year,2000,...,NaN,NaN,26.1,NaN,51.3,38.6 [26.1-51.3],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
13705,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,SEAR,South-East Asia,Country,IDN,Indonesia,Year,2000,...,NaN,NaN,24.0,NaN,41.5,31.7 [24.0-41.5],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z
13751,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive...,text,WPR,Western Pacific,Country,MYS,Malaysia,Year,2000,...,NaN,NaN,20.6,NaN,49.5,33.2 [20.6-49.5],NaN,WHO Global Anaemia estimates. Prevalence of an...,EN,2025-04-28T16:00:00.000Z


In [62]:
GINI = pd.read_csv(r'C:\Users\Leon Pascual\Documents\GitHub\CPE311_Computational-Thinking-with-Python\Final\ADSE\datasets\API_SI.POV.GINI_DS2_en_csv_v2_131994\GINI Index.csv', skiprows=4)
GINI.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,51.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
